#### Version-1

In [35]:

// TableRole
sealed trait TableRole

object TableRole {
  case object Source extends TableRole
  case object Target extends TableRole
}

// Dataset enum (strongly typed)
sealed trait Dataset
object Dataset {
  case object WosCore extends Dataset
  case object WosEsci extends Dataset
  case object Pprn    extends Dataset
}

// Dataset → default context mapping
case class TableContext(
  schema: String,
  suffix: Option[String]
)

// Dataset → default context mapping
object DatasetDefaults {

  def contextFor(dataset: Dataset): TableContext =
    dataset match {
      case Dataset.WosCore =>
        TableContext("gold_wos", Some("woscore"))

      case Dataset.WosEsci =>
        TableContext("gold_wos", None)

      case Dataset.Pprn =>
        TableContext("gold_pprn", None)
    }
}

// Immutable, Job-Scoped TableResolver
final class TableResolver private (
    ctx: TableContext,
    sourceCatalog: String,
    targetCatalog: String,
    sourceEnvironment: String,
    targetEnvironment: String,
    sourceVersion: String,
    targetVersion: String
) {

  def resolve(table: TableMetadata): String = {

    val (catalog, environment, version) = table.role match {
      case TableRole.Source => (sourceCatalog, sourceEnvironment, sourceVersion)
      case TableRole.Target => (targetCatalog, targetEnvironment, targetVersion)
    }

    val tableName =
      if (table.needsSuffix)
        ctx.suffix
          .map(s => s"${table.tableName}_$s")
          .getOrElse(table.tableName)
      else
        table.tableName

    val schema = s"${ctx.schema}_v${version}"

    s"${catalog}_${environment}.${schema}.${tableName}"
  }
}



// Factory (widgets read once)
object TableResolver {

  def forDataset(dataset: Dataset): TableResolver = {

    val ctx = DatasetDefaults.contextFor(dataset)

    new TableResolver(
      ctx             = ctx,
      sourceCatalog   = "ag_content_ims_acs", // getWidget("source_catalog", "ag_content_ims_acs"),
      targetCatalog   = "ag_ra_search_analytics_data", // getWidget("target_catalog", "ag_ra_search_analytics_data"),
      sourceEnvironment   = "prod", // getWidget("source__environment", "prod"),
      targetEnvironment   = "dev", //getWidget("target_environment", "dev")
      sourceVersion   = "1_0_1", // getWidget("source_version", "1_0_1"),
      targetVersion   = "1_0" //    getWidget("target_version", "1_0")
    )
  }

   private def getWidget(name: String, default: String): String = {
      try {
        val value = "test" // dbutils.widgets.get(name)
        if (value == null || value.isEmpty) default else value
      } catch {
        case _: Throwable => default
      }
    }
}


// TableMetadata (Implicit Resolver)
trait TableMetadata {

  def tableName: String
  //def schema: String

  def schema: Option[String] = None
  def role: TableRole
  def needsSuffix: Boolean = false

  final def fullName(implicit resolver: TableResolver): String =
    resolver.resolve(this)

  override def toString: String =
    schema match {
      case Some(s) => s"$s.$tableName"
      case None    => tableName
    }

 /// override def toString: String = s"$schema.$tableName"   // safe, no implicits
}



sealed trait ACS extends TableMetadata {
  final val role: TableRole = TableRole.Source
}

sealed trait UDM extends TableMetadata {
  final val role: TableRole = TableRole.Source
}

sealed trait DAP extends TableMetadata {
  final val role: TableRole = TableRole.Target
}

defined trait TableRole
defined object TableRole
defined trait Dataset
defined object Dataset
defined class TableContext
defined object DatasetDefaults
defined class TableResolver
defined object TableResolver
defined trait TableMetadata
defined trait ACS
defined trait UDM
defined trait DAP


Companions must be defined together; you may wish to use :paste mode for this.


In [ ]:
object ACS {
  case object DArticleTotalCites extends ACS {
    val tableName: String = "DArticleTotalCites"
    override def needsSuffix = true

  }

  case object DAlmaOpenaccess extends ACS {
    val tableName: String = "DAlmaOpenaccess"
    override schema = "gold_entity")
  }

  case object AuthorPublicationLink extends ACS {
    val tableName: String = "AuthorPublicationLink"
  }
}




val datasetList: Seq[Dataset] = Seq(Dataset.WosCore, Dataset.WosEsci, Dataset.Pprn)
datasetList.foreach { t =>

    implicit val resolver: TableResolver =TableResolver.forDataset(Dataset.WosCore)

    println(s"Dataset: $t")
    println("-------------------------")
    println(ACS.DArticleTotalCites.fullName)
    println(ACS.DAlmaOpenaccess.fullName)
    println(ACS.AuthorPublicationLink.fullName)
}

#### Versioon-2

In [1]:

// TableRole
sealed trait TableRole

object TableRole {
  case object Source extends TableRole
  case object Target extends TableRole
}

// Dataset enum (strongly typed)
sealed trait Dataset
object Dataset {
  case object WosCore extends Dataset
  case object WosEsci extends Dataset
  case object Pprn    extends Dataset
}

// Dataset → default context mapping
case class TableContext(
  schema: String,
  suffix: Option[String]
)

// Dataset → default context mapping
object DatasetDefaults {

  def contextFor(dataset: Dataset): TableContext =
    dataset match {
      case Dataset.WosCore =>
        TableContext("gold_wos", Some("woscore"))

      case Dataset.WosEsci =>
        TableContext("gold_wos", None)

      case Dataset.Pprn =>
        TableContext("gold_pprn", None)
    }
}


// Factory (widgets read once)
object TableResolver {

  var ctx: TableContext = TableContext("", None)
  
  def forDataset(dataset: Dataset) = { 
    this.ctx = DatasetDefaults.contextFor(dataset)
  }

  val sourceCatalog   = "ag_content_ims_acs" // getWidget("source_catalog", "ag_content_ims_acs"),
  val targetCatalog   = "ag_ra_search_analytics_data" // getWidget("target_catalog", "ag_ra_search_analytics_data"),
  val sourceEnvironment   = "prod" // getWidget("source__environment", "prod"),
  val targetEnvironment   = "dev" //getWidget("target_environment", "dev")
  val sourceVersion   = "1_0_1" // getWidget("source_version", "1_0_1"),
  val targetVersion   = "1_0" //    getWidget("target_version", "1_0")


  def resolve(table: TableMetadata): String = {

    val (catalog, environment, version) = table.role match {
      case TableRole.Source => (sourceCatalog, sourceEnvironment, sourceVersion)
      case TableRole.Target => (targetCatalog, targetEnvironment, targetVersion)
    }

    val tableName =
      if (table.needsSuffix)
        ctx.suffix
          .map(s => s"${table.tableName}_$s")
          .getOrElse(table.tableName)
      else
        table.tableName
      /*
      val schema = table.schema match {
              case Some(s) => s"${s}_${version}"
              case None    => s"${ctx.schema}_${version}"
            }
      */
    val catalogResolved = if(table.catalog.isEmpty) s"${catalog}" else s"${table.catalog}"
    val schema = if(table.schema.isEmpty) s"${ctx.schema}_${version}" else s"${table.schema}_${version}"

    s"${catalogResolved }_${environment}.${schema}.${tableName}"
  }

   private def getWidget(name: String, default: String): String = {
      try {
        val value = "test" // dbutils.widgets.get(name)
        if (value == null || value.isEmpty) default else value
      } catch {
        case _: Throwable => default
      }
    }
}


// TableMetadata (Implicit Resolver)
trait TableMetadata {

  def tableName: String
  def schema: String = ""
  def catalog: String = ""
  // def schema: Option[String] = None
  def role: TableRole
  def needsSuffix: Boolean = false

  def fullName: String = TableResolver.resolve(this)

  override def toString: String = fullName
    
 /// override def toString: String = s"$schema.$tableName"   // safe, no implicits
}



sealed trait ACS extends TableMetadata {
  final val role: TableRole = TableRole.Source
}

sealed trait UDM extends TableMetadata {
  final val role: TableRole = TableRole.Source
	override val catalog = "ag_content_ims_udm"
}

sealed trait DAP extends TableMetadata {
  final val role: TableRole = TableRole.Target
}

defined trait TableRole
defined object TableRole
defined trait Dataset
defined object Dataset
defined class TableContext
defined object DatasetDefaults
defined object TableResolver
defined trait TableMetadata
defined trait ACS
defined trait UDM
defined trait DAP


#### Define table - ACS

In [2]:
// AUTO-GENERATED FILE — DO NOT EDIT
// Generated by Python

object ACS {

	case object AddressPublicationLink extends ACS {
		val tableName = "address_publication_link"
	}

	case object AuthorPublicationLink extends ACS {
		val tableName = "author_publication_link"
	}

	case object CategoryPublicationLink extends ACS {
		val tableName = "category_publication_link"
	}

	case object CountryPublicationLink extends ACS {
		val tableName = "country_publication_link"
	}

	case object CountryTerritoryLink extends ACS {
		val tableName = "country_territory_link"
	}

	case object DAlcCompleteLabels extends ACS {
		val tableName = "d_alc_complete_labels"
	}

	case object DAlmaOpenaccess extends ACS {
		val tableName = "d_alma_openaccess"
		override val schema = "gold_entity"
	}

	case object DAlmaSubscriptions extends ACS {
		val tableName = "d_alma_subscriptions"
		override val schema = "gold_entity"
	}

	case object DArticleFlag extends ACS {
		val tableName = "d_article_flag"
		override val needsSuffix = true
	}

	case object DArticleIdentifiers extends ACS {
		val tableName = "d_article_identifiers"
	}

	case object DArticleMetrics extends ACS {
		val tableName = "d_article_metrics"
		override val needsSuffix = true
	}

	case object DArticleTotalCites extends ACS {
		val tableName = "d_article_total_cites"
		override val needsSuffix = true
	}

	case object DArticleType extends ACS {
		val tableName = "d_article_type"
	}

	case object DArticleTypePrecedence extends ACS {
		val tableName = "d_article_type_precedence"
	}

	case object DAuthor extends ACS {
		val tableName = "d_author"
	}

	case object DCategory extends ACS {
		val tableName = "d_category"
	}

	case object DCitation extends ACS {
		val tableName = "d_citation"
	}

	case object DCitationFlag extends ACS {
		val tableName = "d_citation_flag"
	}

	case object DCitationPatent extends ACS {
		val tableName = "d_citation_patent"
	}

	case object DCitationPatentFlag extends ACS {
		val tableName = "d_citation_patent_flag"
	}

	case object DCountry extends ACS {
		val tableName = "d_country"
	}

	case object DDaisngRankedAffiliation extends ACS {
		val tableName = "d_daisng_ranked_affiliation"
		override val needsSuffix = true
	}

	case object DDiidwArticlekey extends ACS {
		val tableName = "d_diidw_articlekey"
	}

	case object DDiidwInventors extends ACS {
		val tableName = "d_diidw_inventors"
	}

	case object DDoctype extends ACS {
		val tableName = "d_doctype"
	}

	case object DDomesticCollab extends ACS {
		val tableName = "d_domestic_collab"
	}

	case object DDomesticInternationalCollab extends ACS {
		val tableName = "d_domestic_international_collab"
		override val needsSuffix = true
	}

	case object DDomesticInternationalCollabtype extends ACS {
		val tableName = "d_domestic_international_collabtype"
	}

	case object DEdition extends ACS {
		val tableName = "d_edition"
	}

	case object DEmtCompleteLabels extends ACS {
		val tableName = "d_emt_complete_labels"
	}

	case object DEsiArticle extends ACS {
		val tableName = "d_esi_article"
		override val schema = "gold_entity"
	}

	case object DEsiAuthorIndicator extends ACS {
		val tableName = "d_esi_author_indicator"
		override val schema = "gold_entity"
	}

	case object DEsiCountryIndicator extends ACS {
		val tableName = "d_esi_country_indicator"
		override val schema = "gold_entity"
	}

	case object DEsiInstitutionIndicator extends ACS {
		val tableName = "d_esi_institution_indicator"
		override val schema = "gold_entity"
	}

	case object DEsiJournalIndicator extends ACS {
		val tableName = "d_esi_journal_indicator"
		override val schema = "gold_entity"
	}

	case object DEsiPapers extends ACS {
		val tableName = "d_esi_papers"
		override val schema = "gold_entity"
	}

	case object DFundingOrganization extends ACS {
		val tableName = "d_funding_organization"
		override val schema = "gold_entity"
	}

	case object DGrantDataitem extends ACS {
		val tableName = "d_grant_dataitem"
	}

	case object DGrantmasterDataitem extends ACS {
		val tableName = "d_grantmaster_dataitem"
		override val schema = "gold_entity"
	}

	case object DGrantmasterGrantid extends ACS {
		val tableName = "d_grantmaster_grantid"
		override val schema = "gold_entity"
	}

	case object DIpOrganisation extends ACS {
		val tableName = "d_ip_organisation"
		override val schema = "gold_entity"
	}

	case object DIpSubject extends ACS {
		val tableName = "d_ip_subject"
		override val schema = "gold_entity"
	}

	case object DIpVariableData extends ACS {
		val tableName = "d_ip_variable_data"
		override val schema = "gold_entity"
	}

	case object DIpVariables extends ACS {
		val tableName = "d_ip_variables"
		override val schema = "gold_entity"
	}

	case object DJcrJournals extends ACS {
		val tableName = "d_jcr_journals"
		override val schema = "gold_entity"
	}

	case object DJournal extends ACS {
		val tableName = "d_journal"
	}

	case object DJournalAcs extends ACS {
		val tableName = "d_journal_acs"
	}

	case object DLocation extends ACS {
		val tableName = "d_location"
	}

	case object DNuts extends ACS {
		val tableName = "d_nuts"
		override val schema = "gold_entity"
	}

	case object DNutsCode extends ACS {
		val tableName = "d_nuts_code"
		override val schema = "gold_entity"
	}

	case object DOaArticle extends ACS {
		val tableName = "d_oa_article"
	}

	case object DOrcidRid extends ACS {
		val tableName = "d_orcid_rid"
		override val schema = "gold_entity"
	}

	case object DOrganization extends ACS {
		val tableName = "d_organization"
	}

	case object DOrganizationCitationCount extends ACS {
		val tableName = "d_organization_citation_count"
		override val needsSuffix = true
	}

	case object DOrganizationCitationFlag extends ACS {
		val tableName = "d_organization_citation_flag"
	}

	case object DOrgmaster extends ACS {
		val tableName = "d_orgmaster"
		override val schema = "gold_entity"
	}

	case object DOrgmasterAssociation extends ACS {
		val tableName = "d_orgmaster_association"
		override val schema = "gold_entity"
	}

	case object DParentChild extends ACS {
		val tableName = "d_parent_child"
		override val schema = "gold_entity"
	}

	case object DPublicationSourceCitationCount extends ACS {
		val tableName = "d_publication_source_citation_count"
		override val needsSuffix = true
	}

	case object DPublicationSourceCitationFlag extends ACS {
		val tableName = "d_publication_source_citation_flag"
	}

	case object DPublisher extends ACS {
		val tableName = "d_publisher"
	}

	case object DResearcherCitationCount extends ACS {
		val tableName = "d_researcher_citation_count"
		override val needsSuffix = true
	}

	case object DResearcherCitationsFlag extends ACS {
		val tableName = "d_researcher_citations_flag"
	}

	case object DSpmaster extends ACS {
		val tableName = "d_spmaster"
		override val schema = "gold_entity"
	}

	case object DState extends ACS {
		val tableName = "d_state"
	}

	case object DTerritory extends ACS {
		val tableName = "d_territory"
	}

	case object DYearwiseCitationCount extends ACS {
		val tableName = "d_yearwise_citation_count"
		override val needsSuffix = true
	}

	case object FCategory extends ACS {
		val tableName = "f_category"
		override val needsSuffix = true
	}

	case object FIpInstitution extends ACS {
		val tableName = "f_ip_institution"
		override val schema = "gold_entity"
	}

	case object FIpInstitutionReputation extends ACS {
		val tableName = "f_ip_institution_reputation"
		override val schema = "gold_entity"
	}

	case object FJcrJournal extends ACS {
		val tableName = "f_jcr_journal"
		override val schema = "gold_entity"
	}

	case object FJcrJournalCategory extends ACS {
		val tableName = "f_jcr_journal_category"
		override val schema = "gold_entity"
	}

	case object FJcrJournalJci extends ACS {
		val tableName = "f_jcr_journal_jci"
		override val schema = "gold_entity"
	}

	case object FPublication extends ACS {
		val tableName = "f_publication"
	}

	case object FundingOrgPublicationLink extends ACS {
		val tableName = "funding_org_publication_link"
		override val schema = "gold_entity"
	}

	case object GrantFundingOrganizationLink extends ACS {
		val tableName = "grant_funding_organization_link"
	}

	case object GrantmasterOrgmasterLink extends ACS {
		val tableName = "grantmaster_orgmaster_link"
		override val schema = "gold_entity"
	}

	case object JournalAcsPublicationLink extends ACS {
		val tableName = "journal_acs_publication_link"
	}

	case object JournalPublicationLink extends ACS {
		val tableName = "journal_publication_link"
	}

	case object JournalPublisherLink extends ACS {
		val tableName = "journal_publisher_link"
		override val schema = "gold_entity"
	}

	case object NutsPublicationLink extends ACS {
		val tableName = "nuts_publication_link"
	}

	case object OrcidRidPublicationLink extends ACS {
		val tableName = "orcid_rid_publication_link"
		override val schema = "gold_entity"
	}

	case object OrgmasterHierarchyPublicationLink extends ACS {
		val tableName = "orgmaster_hierarchy_publication_link"
		override val schema = "gold_entity"
	}

	case object OrgmasterPublicationLink extends ACS {
		val tableName = "orgmaster_publication_link"
		override val schema = "gold_entity"
	}

	case object PublicationDoctypeLink extends ACS {
		val tableName = "publication_doctype_link"
	}

	case object PublisherPublicationLink extends ACS {
		val tableName = "publisher_publication_link"
	}

	case object ReprintAddressPublicationLink extends ACS {
		val tableName = "reprint_address_publication_link"
	}

	case object SpmasterPublicationLink extends ACS {
		val tableName = "spmaster_publication_link"
		override val schema = "gold_entity"
	}

	case object StatePublicationLink extends ACS {
		val tableName = "state_publication_link"
	}

	case object TerritoryPublicationLink extends ACS {
		val tableName = "territory_publication_link"
	}

}


defined object ACS


Companions must be defined together; you may wish to use :paste mode for this.


In [3]:

  
val datasetList: Seq[Dataset] = Seq(Dataset.WosCore,  Dataset.Pprn, Dataset.WosEsci)

datasetList.foreach { t =>

    TableResolver.forDataset(t)
    println("-------------------------")
    println(s"Dataset: $t")
    println(ACS.DArticleTotalCites)
    println(ACS.DAlmaOpenaccess)
    println(ACS.AuthorPublicationLink)

}

-------------------------
Dataset: WosCore
ag_content_ims_acs_prod.gold_wos_1_0_1.d_article_total_cites_woscore
ag_content_ims_acs_prod.gold_entity_1_0_1.d_alma_openaccess
ag_content_ims_acs_prod.gold_wos_1_0_1.author_publication_link
-------------------------
Dataset: Pprn
ag_content_ims_acs_prod.gold_pprn_1_0_1.d_article_total_cites
ag_content_ims_acs_prod.gold_entity_1_0_1.d_alma_openaccess
ag_content_ims_acs_prod.gold_pprn_1_0_1.author_publication_link
-------------------------
Dataset: WosEsci
ag_content_ims_acs_prod.gold_wos_1_0_1.d_article_total_cites
ag_content_ims_acs_prod.gold_entity_1_0_1.d_alma_openaccess
ag_content_ims_acs_prod.gold_wos_1_0_1.author_publication_link


datasetList = List(WosCore, Pprn, WosEsci)


List(WosCore, Pprn, WosEsci)

In [128]:

def runWos(): Unit = {


  TableResolver.forDataset(Dataset.WosCore)   
  
  println(ACS.DArticleTotalCites)
  println(ACS.DAlmaOpenaccess)
  println(ACS.AuthorPublicationLink)

}

def runEsci(): Unit = {

  TableResolver.forDataset(Dataset.WosEsci)  
  
  println(ACS.DArticleTotalCites)
  println(ACS.DAlmaOpenaccess)
  println(ACS.AuthorPublicationLink)

}

def runPprn(): Unit = {
 
  TableResolver.forDataset(Dataset.Pprn)  
  
  println(ACS.DArticleTotalCites)
  println(ACS.DAlmaOpenaccess)
  println(ACS.AuthorPublicationLink)

}

println("---------------WOS-CORE--------------------------")
runWos()

println("---------------WOS-SESCI--------------------------")
runEsci()

println("---------------PPRN--------------------------")
runPprn()

runWos: ()Unit
runEsci: ()Unit
runPprn: ()Unit


---------------WOS-CORE--------------------------
ag_content_ims_acs_prod.gold_pprn_1_0_1.d_article_total_cites
ag_content_ims_acs_prod.gold_entity_1_0_1.d_alma_openaccess
ag_content_ims_acs_prod.gold_pprn_1_0_1.author_publication_link
---------------WOS-SESCI--------------------------
ag_content_ims_acs_prod.gold_pprn_1_0_1.d_article_total_cites
ag_content_ims_acs_prod.gold_entity_1_0_1.d_alma_openaccess
ag_content_ims_acs_prod.gold_pprn_1_0_1.author_publication_link
---------------PPRN--------------------------
ag_content_ims_acs_prod.gold_pprn_1_0_1.d_article_total_cites
ag_content_ims_acs_prod.gold_entity_1_0_1.d_alma_openaccess
ag_content_ims_acs_prod.gold_pprn_1_0_1.author_publication_link


#### Define table - DAP

In [126]:

// AUTO-GENERATED FILE — DO NOT EDIT
// Generated by Python

object DAP {

	case object Alma extends DAP {
		val tableName = "alma"
		override val schema = "dap_entity_wos"
	}

	case object ApArticle extends DAP {
		val tableName = "ap_article"
		override val schema = "dap_entity_wos"
	}

	case object ApAuthorprofile extends DAP {
		val tableName = "ap_authorprofile"
		override val schema = "dap_work"
	}

	case object ApCategory extends DAP {
		val tableName = "ap_category"
		override val schema = "dap_work"
	}

	case object ApDept extends DAP {
		val tableName = "ap_dept"
		override val schema = "dap_work"
	}

	case object ApFunder extends DAP {
		val tableName = "ap_funder"
		override val schema = "dap_work"
	}

	case object ApMetric extends DAP {
		val tableName = "ap_metric"
		override val schema = "dap_work"
	}

	case object ApOrg extends DAP {
		val tableName = "ap_org"
		override val schema = "dap_work"
	}

	case object ApRigion extends DAP {
		val tableName = "ap_rigion"
		override val schema = "dap_work"
	}

	case object ArticleNormalizedMetrics extends DAP {
		val tableName = "article_normalized_metrics"
		override val schema = "dap_metrics_wos"
		override val needsSuffix = true
	}

	case object Authorprofile extends DAP {
		val tableName = "authorprofile"
		override val schema = "dap_entity_wos"
	}

	case object AuthorprofileWos extends DAP {
		val tableName = "authorprofile_wos"
		override val schema = "dap_prod_core"
	}

	case object CateAuthorprofileEileen extends DAP {
		val tableName = "cate_authorprofile_eileen"
		override val schema = "dap_work"
	}

	case object CateWosEileen extends DAP {
		val tableName = "cate_wos_eileen"
		override val schema = "dap_work"
	}

	case object Category extends DAP {
		val tableName = "category"
		override val schema = "dap_entity_wos"
	}

	case object CategoryArticleMetrics extends DAP {
		val tableName = "category_article_metrics"
		override val schema = "dap_entity_wos"
	}

	case object CategoryMetrics extends DAP {
		val tableName = "category_metrics"
		override val schema = "dap_metrics_wos"
	}

	case object CustDatasetDefault extends DAP {
		val tableName = "cust_dataset_default"
		override val schema = "dap_reference"
	}

	case object Custom extends DAP {
		val tableName = "custom"
		override val schema = "dap_prod_core"
	}

	case object DapCheckpointHistory extends DAP {
		val tableName = "dap_checkpoint_history"
		override val schema = "dap_ops"
	}

	case object DapCheckpoints extends DAP {
		val tableName = "dap_checkpoints"
		override val schema = "dap_ops"
	}

	case object DapPipelineRegistry extends DAP {
		val tableName = "dap_pipeline_registry"
		override val schema = "dap_ops"
	}

	case object DapPipelineTaskRuns extends DAP {
		val tableName = "dap_pipeline_task_runs"
		override val schema = "dap_ops"
	}

	case object DapPipelineTaskUpstream extends DAP {
		val tableName = "dap_pipeline_task_upstream"
		override val schema = "dap_ops"
	}

	case object DapWatermarkHistory extends DAP {
		val tableName = "dap_watermark_history"
		override val schema = "dap_ops"
	}

	case object DapWatermarks extends DAP {
		val tableName = "dap_watermarks"
		override val schema = "dap_ops"
	}

	case object Datasets extends DAP {
		val tableName = "datasets"
		override val schema = "dap_reference"
	}

	case object DeptArticle extends DAP {
		val tableName = "dept_article"
		override val schema = "dap_entity_wos"
	}

	case object EnResearchTopics extends DAP {
		val tableName = "en_research_topics"
		override val schema = "dap_metrics_wos"
	}

	case object EnSocietalFacet extends DAP {
		val tableName = "en_societal_facet"
		override val schema = "dap_metrics_wos"
	}

	case object EnSocietalImpact extends DAP {
		val tableName = "en_societal_impact"
		override val schema = "dap_metrics_wos"
	}

	case object FunderCatalog extends DAP {
		val tableName = "funder_catalog"
		override val schema = "dap_reference"
	}

	case object FundingAgency extends DAP {
		val tableName = "funding_agency"
		override val schema = "dap_entity_wos"
	}

	case object FundingAgencyArticleMetrics extends DAP {
		val tableName = "funding_agency_article_metrics"
		override val schema = "dap_entity_wos"
	}

	case object Grant extends DAP {
		val tableName = "grant"
		override val schema = "dap_prod_core"
	}

	case object Grants extends DAP {
		val tableName = "grants"
		override val schema = "dap_prod_core"
	}

	case object GrantsRiAuthorprofile extends DAP {
		val tableName = "grants_ri_authorprofile"
		override val schema = "dap_grant"
	}

	case object GrantsRiGrantFunder extends DAP {
		val tableName = "grants_ri_grant_funder"
		override val schema = "dap_grant"
	}

	case object GrantsRiGrantOrg extends DAP {
		val tableName = "grants_ri_grant_org"
		override val schema = "dap_grant"
	}

	case object GrantsRiGrantUtFunder extends DAP {
		val tableName = "grants_ri_grant_ut_funder"
		override val schema = "dap_grant"
	}

	case object GrantsRiGrantUtOrg extends DAP {
		val tableName = "grants_ri_grant_ut_org"
		override val schema = "dap_grant"
	}

	case object GrantsRiGrantsArticle extends DAP {
		val tableName = "grants_ri_grants_article"
		override val schema = "dap_grant"
	}

	case object GrantsRiGrantsItems extends DAP {
		val tableName = "grants_ri_grants_items"
		override val schema = "dap_work"
	}

	case object GrantsRiGrantsUts extends DAP {
		val tableName = "grants_ri_grants_uts"
		override val schema = "dap_grant"
	}

	case object GrantsRiGrantsUtsEileen extends DAP {
		val tableName = "grants_ri_grants_uts_eileen"
		override val schema = "dap_work"
	}

	case object GrantsRiGrantsUtsYangming extends DAP {
		val tableName = "grants_ri_grants_uts_yangming"
		override val schema = "dap_work"
	}

	case object GrantsRiResearchTopic extends DAP {
		val tableName = "grants_ri_research_topic"
		override val schema = "dap_grant"
	}

	case object GrantsRiUnited extends DAP {
		val tableName = "grants_ri_united"
		override val schema = "dap_prod_core"
	}

	case object GrantsRiUnitedOrignal extends DAP {
		val tableName = "grants_ri_united_orignal"
		override val schema = "dap_prod_core"
	}

	case object IncitesAuthorprofile extends DAP {
		val tableName = "incites_authorprofile"
		override val schema = "dap_work"
	}

	case object IncitesCategory extends DAP {
		val tableName = "incites_category"
		override val schema = "dap_work"
	}

	case object IncitesDept extends DAP {
		val tableName = "incites_dept"
		override val schema = "dap_work"
	}

	case object IncitesFunder extends DAP {
		val tableName = "incites_funder"
		override val schema = "dap_work"
	}

	case object IncitesFunderNames extends DAP {
		val tableName = "incites_funder_names"
		override val schema = "dap_work"
	}

	case object IncitesMetric extends DAP {
		val tableName = "incites_metric"
		override val schema = "dap_work"
	}

	case object IncitesOrg extends DAP {
		val tableName = "incites_org"
		override val schema = "dap_work"
	}

	case object IncitesRiCategory extends DAP {
		val tableName = "incites_ri_category"
		override val schema = "dap_work"
	}

	case object IncitesRiFundingOrganizations extends DAP {
		val tableName = "incites_ri_funding_organizations"
		override val schema = "dap_grant"
	}

	case object IncitesRiOrgGrants extends DAP {
		val tableName = "incites_ri_org_grants"
		override val schema = "dap_grant"
	}

	case object IncitesRiOrganizations extends DAP {
		val tableName = "incites_ri_organizations"
		override val schema = "dap_grant"
	}

	case object IncitesRigion extends DAP {
		val tableName = "incites_rigion"
		override val schema = "dap_work"
	}

	case object IncitesWos extends DAP {
		val tableName = "incites_wos"
		override val schema = "dap_prod_core"
	}

	case object IncitesWosCategory extends DAP {
		val tableName = "incites_wos_category"
		override val schema = "dap_work"
	}

	case object JcrMetrics extends DAP {
		val tableName = "jcr_metrics"
		override val schema = "dap_entity_wos"
	}

	case object JcrMetricsMore extends DAP {
		val tableName = "jcr_metrics_more"
		override val schema = "dap_entity_wos"
	}

	case object Journal extends DAP {
		val tableName = "journal"
		override val schema = "dap_entity_wos"
	}

	case object NutsCodeRegion extends DAP {
		val tableName = "nuts_code_region"
		override val schema = "dap_reference"
	}

	case object OrgArticle extends DAP {
		val tableName = "org_article"
		override val schema = "dap_entity_wos"
	}

	case object OrgLeadlag extends DAP {
		val tableName = "org_leadlag"
		override val schema = "dap_prod_core"
	}

	case object OrgRanking extends DAP {
		val tableName = "org_ranking"
		override val schema = "dap_prod_core"
	}

	case object OrgTopParentMapping extends DAP {
		val tableName = "org_top_parent_mapping"
		override val schema = "dap_work"
	}

	case object Organization extends DAP {
		val tableName = "organization"
		override val schema = "dap_entity_wos"
	}

	case object OrgcIp extends DAP {
		val tableName = "orgc_ip"
		override val schema = "dap_entity_wos"
	}

	case object PatentGrantedApplication extends DAP {
		val tableName = "patent_granted_application"
		override val schema = "dap_work"
	}

	case object PatentOrg extends DAP {
		val tableName = "patent_org"
		override val schema = "dap_work"
	}

	case object Patents extends DAP {
		val tableName = "patents"
		override val schema = "dap_prod_core"
	}

	case object Pprn extends DAP {
		val tableName = "pprn"
		override val schema = "dap_entity_pprn"
	}

	case object Region extends DAP {
		val tableName = "region"
		override val schema = "dap_entity_wos"
	}

	case object RegionArticleMetrics extends DAP {
		val tableName = "region_article_metrics"
		override val schema = "dap_entity_wos"
	}

	case object RevisedPublisherUnification extends DAP {
		val tableName = "revised_publisher_unification"
		override val schema = "dap_reference"
	}

	case object RiGrant extends DAP {
		val tableName = "ri_grant"
		override val schema = "dap_prod_core"
	}

	case object RiGrantOriginal extends DAP {
		val tableName = "ri_grant_original"
		override val schema = "dap_prod_core"
	}

	case object RiWos extends DAP {
		val tableName = "ri_wos"
		override val schema = "dap_prod_core"
	}

	case object SimBaselineOrgLeadlag extends DAP {
		val tableName = "sim_baseline_org_leadlag"
		override val schema = "dap_prod_core"
	}

	case object SimBaselineOrgRanking extends DAP {
		val tableName = "sim_baseline_org_ranking"
		override val schema = "dap_prod_core"
	}

	case object SimMetAcadOrganizationCount extends DAP {
		val tableName = "sim_met_acad_organization_count"
		override val schema = "dap_work"
	}

	case object SimMetInstitutionCountByPubCount extends DAP {
		val tableName = "sim_met_institution_count_by_pub_count"
		override val schema = "dap_work"
	}

	case object SimMetInstitutionCountByRatio extends DAP {
		val tableName = "sim_met_institution_count_by_ratio"
		override val schema = "dap_work"
	}

	case object SimMetMetricTotal extends DAP {
		val tableName = "sim_met_metric_total"
		override val schema = "dap_work"
	}

	case object SimMetMinPubCountPerQuantile extends DAP {
		val tableName = "sim_met_min_pub_count_per_quantile"
		override val schema = "dap_work"
	}

	case object SimMetMinRatioPerQuantile extends DAP {
		val tableName = "sim_met_min_ratio_per_quantile"
		override val schema = "dap_work"
	}

	case object SimMetOrganizationMetrics extends DAP {
		val tableName = "sim_met_organization_metrics"
		override val schema = "dap_work"
	}

	case object SimMetOrganizationMetricsFull extends DAP {
		val tableName = "sim_met_organization_metrics_full"
		override val schema = "dap_work"
	}

	case object SimMetPercentageAboveThresholdByPubCount extends DAP {
		val tableName = "sim_met_percentage_above_threshold_by_pub_count"
		override val schema = "dap_work"
	}

	case object SimMetPercentageAboveThresholdByPubCountWithThreshold extends DAP {
		val tableName = "sim_met_percentage_above_threshold_by_pub_count_with_threshold"
		override val schema = "dap_work"
	}

	case object SimMetPercentageAboveThresholdByRatioWithThreshold extends DAP {
		val tableName = "sim_met_percentage_above_threshold_by_ratio_with_threshold"
		override val schema = "dap_work"
	}

	case object SimMetPossibleThresholds extends DAP {
		val tableName = "sim_met_possible_thresholds"
		override val schema = "dap_work"
	}

	case object SimMetSizeNormalizedIndicators extends DAP {
		val tableName = "sim_met_size_normalized_indicators"
		override val schema = "dap_work"
	}

	case object SimMetThresholds extends DAP {
		val tableName = "sim_met_thresholds"
		override val schema = "dap_work"
	}

	case object SimMetUtCountByOrganization extends DAP {
		val tableName = "sim_met_ut_count_by_organization"
		override val schema = "dap_work"
	}

	case object SimMetUtExploded extends DAP {
		val tableName = "sim_met_ut_exploded"
		override val schema = "dap_work"
	}

	case object SimMetUtProcessed extends DAP {
		val tableName = "sim_met_ut_processed"
		override val schema = "dap_work"
	}

	case object SimSrcDiidwAcadOrganizationCount extends DAP {
		val tableName = "sim_src_diidw_acad_organization_count"
		override val schema = "dap_work"
	}

	case object SimSrcDiidwDocsExploded extends DAP {
		val tableName = "sim_src_diidw_docs_exploded"
		override val schema = "dap_work"
	}

	case object SimSrcDiidwDocsProcessed extends DAP {
		val tableName = "sim_src_diidw_docs_processed"
		override val schema = "dap_work"
	}

	case object SimSrcDiidwDocsRaw extends DAP {
		val tableName = "sim_src_diidw_docs_raw"
		override val schema = "dap_work"
	}

	case object SimSrcDiidwUtCountByOrganization extends DAP {
		val tableName = "sim_src_diidw_ut_count_by_organization"
		override val schema = "dap_work"
	}

	case object SimSrcWosAcadOrganizationCount extends DAP {
		val tableName = "sim_src_wos_acad_organization_count"
		override val schema = "dap_work"
	}

	case object SimSrcWosDocsExploded extends DAP {
		val tableName = "sim_src_wos_docs_exploded"
		override val schema = "dap_work"
	}

	case object SimSrcWosDocsProcessed extends DAP {
		val tableName = "sim_src_wos_docs_processed"
		override val schema = "dap_work"
	}

	case object SimSrcWosDocsRaw extends DAP {
		val tableName = "sim_src_wos_docs_raw"
		override val schema = "dap_work"
	}

	case object SimSrcWosOrganizationCombination extends DAP {
		val tableName = "sim_src_wos_organization_combination"
		override val schema = "dap_work"
	}

	case object SimSrcWosOrganizations extends DAP {
		val tableName = "sim_src_wos_organizations"
		override val schema = "dap_work"
	}

	case object SimSrcWosUtCountByOrganization extends DAP {
		val tableName = "sim_src_wos_ut_count_by_organization"
		override val schema = "dap_work"
	}

	case object TempFullPprn extends DAP {
		val tableName = "temp_full_pprn"
		override val schema = "dap_work"
	}

	case object TempFullWos extends DAP {
		val tableName = "temp_full_wos"
		override val schema = "dap_work"
	}

	case object Wos extends DAP {
		val tableName = "wos"
		override val schema = "dap_entity_wos"
	}

	case object WosAlma extends DAP {
		val tableName = "wos_alma"
		override val schema = "dap_prod_core"
	}

	case object WosAuthorProfile extends DAP {
		val tableName = "wos_author_profile"
		override val schema = "dap_prod_core"
	}

	case object WosAuthorprofile extends DAP {
		val tableName = "wos_authorprofile"
		override val schema = "dap_prod_core"
	}

	case object WosDict extends DAP {
		val tableName = "wos_dict"
		override val schema = "dap_prod_core"
	}

	case object WosMetadata extends DAP {
		val tableName = "wos_metadata"
		override val schema = "dap_prod_core"
	}

	case object WosOrgcIp extends DAP {
		val tableName = "wos_orgc_ip"
		override val schema = "dap_prod_core"
	}

	case object WosOriginal extends DAP {
		val tableName = "wos_original"
		override val schema = "dap_entity_wos"
	}

	case object WosTile extends DAP {
		val tableName = "wos_tile"
		override val schema = "dap_prod_core"
	}

	case object Wosri extends DAP {
		val tableName = "wosri"
		override val schema = "dap_prod_core"
	}

	case object WosriFinal extends DAP {
		val tableName = "wosri_final"
		override val schema = "dap_prod_core"
	}

	case object WosriMid extends DAP {
		val tableName = "wosri_mid"
		override val schema = "dap_prod_core"
	}

	case object WosriOriginal extends DAP {
		val tableName = "wosri_original"
		override val schema = "dap_prod_core"
	}

}


defined object DAP


<console>:636: warning: Generated class DAP$WosAuthorprofile$ differs only in case from DAP$WosAuthorProfile$.
  Such classes will overwrite one another on case-insensitive filesystems.
       	case object WosAuthorprofile extends DAP {
                           ^
Companions must be defined together; you may wish to use :paste mode for this.


In [127]:


  println(DAP.Alma)
  println(DAP.ApArticle)
  println(DAP.IncitesRiOrgGrants)



ag_ra_search_analytics_data_dev.dap_entity_wos_1_0.alma
ag_ra_search_analytics_data_dev.dap_entity_wos_1_0.ap_article
ag_ra_search_analytics_data_dev.dap_grant_1_0.incites_ri_org_grants


#### Define table - UDM

In [136]:
// AUTO-GENERATED FILE — DO NOT EDIT
// Generated by Python

object UDM {

	case object GrantsTopic extends UDM {
		val tableName = "grants_topic"
		override val schema = "topic_model"
	}

	case object ItemTopic extends UDM {
		val tableName = "item_topic"
		override val schema = "topic_model"
	}

	case object PatentOrgPacoMapping extends UDM {
		val tableName = "patent_org_paco_mapping"
		override val schema = "patent_org_relationship"
	}

	case object PatentTopic extends UDM {
		val tableName = "patent_topic"
		override val schema = "topic_model"
	}

	case object ProfileGrantRelation extends UDM {
		val tableName = "profile_grant_relation"
		override val schema = "researcher_grants_relationship"
	}

	case object TaxonomyDetails extends UDM {
		val tableName = "taxonomy_details"
		override val schema = "topic_model"
	}

}


defined object UDM


Companions must be defined together; you may wish to use :paste mode for this.


In [137]:
    println(UDM.GrantsTopic)
    println(UDM.ProfileGrantRelation)
    println(UDM.ItemTopic)

ag_content_ims_udm_prod.topic_model_1_0_1.grants_topic
ag_content_ims_udm_prod.researcher_grants_relationship_1_0_1.profile_grant_relation
ag_content_ims_udm_prod.topic_model_1_0_1.item_topic
